In [ ]:
#This code was helped by Justing Robert
#To run need to do it in Google Colab
"""
First code of setting up the CNN, we still need different things like downloading the data
and setting up in the right way (path) so that the CNN can access it. We also need later on to do functions to actually test the CNN in a way
that we can see the results. To do this I used the tensor flow last thing and I used the last CNN program that we worked on. If you have any questions
please text me

I moved our imports to a seperate block  so that we could run them before testing any code. - Justin
"""


# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #For processing the images so they can be run through the network
from tensorflow.keras.models import Sequential #Needed for building the network nodes
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense #Needed for building the Coonvultional layers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import auth, drive #For accessing the database through drive
auth.authenticate_user()
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Define paths to the training and testing data directories

datasets = "/content/drive/MyDrive/410project/dataset/"

train_data_dir = os.path.join(datasets, "training")
test_data_dir = os.path.join(datasets, "validation")

print(os.listdir(train_data_dir))
print(os.listdir(test_data_dir))
#Incase we need access to specific cat breed folders I will make variables with directories to each one
"""See training folders"""
training_persion = os.path.join(train_data_dir, "persian")
training_abyssian = os.path.join(train_data_dir, "abyssian")
#training_munchkin = os.path.join(train_data_dir, "munchkin")
training_toyger = os.path.join(train_data_dir, "toyger")

"""See testing folders"""
#testing_munchkin = os.path.join(test_data_dir, "munchkin")
testing_toyger = os.path.join(test_data_dir, "toyger")
testing_abyssian = os.path.join(test_data_dir, "abyssian")
testing_persian = os.path.join(test_data_dir, "persian")

['persian', 'abyssian', 'toyger']
['toyger', 'abyssian', 'persian']


In [ ]:

# Define image dimensions and batch size
img_width, img_height = 250, 250
batch_size = 32 # determes the number of samples processed by the model in each training iteration

# Create data generors with augmentation for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Rescale pixel values to [0,1]
    shear_range=0.2,            # Shearing transformation
    zoom_range=0.2,             # Zooming transformation
    horizontal_flip=True)       # Horizontal flipping

test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values for testing data

In [ ]:
"""
train_generator is an object that will continuously generate batches of training data from the specified directory
with images resized to  dimensions and one encoded class
labels suitable for multi-class classification tasks.

- train_datagen is a instance of the ImageDataGenerator used to prepocess the images
- flow_from_directory method of the ImageDataGenerator class
- class_mode specifies the type of labels returned by the generator
"""
train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
    class_mode='categorical')  # Use categorical labels for multi-class classification

validation_generator = test_datagen.flow_from_directory(
    test_data_dir, target_size=(img_width, img_height), batch_size=batch_size,
    class_mode='categorical')

Found 1077 images belonging to 3 classes.
Found 448 images belonging to 3 classes.


In [ ]:
# Define the CNN model
model = Sequential()  # Create a sequential model

# Adding Convolutional layers with MaxPooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 124, 124, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 61, 61, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 29, 29, 32)        0

In [ ]:
# Flatten the output from Convolutional layers for input to Dense layers
model.add(Flatten())

# Add Dense layers
model.add(Dense(512, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))  # Output layer

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 124, 124, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 61, 61, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 29, 29, 32)        0

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Number of steps per epoch,
    epochs=16,                                              # Number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)  # Number of validation steps

Epoch 1/16
31/33 [===========================>..] - ETA: 8s - loss: 1.0144 - accuracy: 0.4437 

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


33/33 [==============================] - 166s 5s/step - loss: 1.0119 - accuracy: 0.4442 - val_loss: 0.9119 - val_accuracy: 0.5312
Epoch 2/16
33/33 [==============================] - 160s 5s/step - loss: 0.9057 - accuracy: 0.5377 - val_loss: 0.8085 - val_accuracy: 0.5826
Epoch 3/16
33/33 [==============================] - 162s 5s/step - loss: 0.8241 - accuracy: 0.6044 - val_loss: 0.7515 - val_accuracy: 0.6987
Epoch 4/16
33/33 [==============================] - 158s 5s/step - loss: 0.6835 - accuracy: 0.7140 - val_loss: 0.7703 - val_accuracy: 0.6853
Epoch 5/16
33/33 [==============================] - 161s 5s/step - loss: 0.6280 - accuracy: 0.7340 - val_loss: 0.6202 - val_accuracy: 0.7165
Epoch 6/16
33/33 [==============================] - 158s 5s/step - loss: 0.6095 - accuracy: 0.7512 - val_loss: 0.6197 - val_accuracy: 0.7299
Epoch 7/16
33/33 [==============================] - 160s 5s/step - loss: 0.5186 - accuracy: 0.7998 - val_loss: 0.4859 - val_accuracy: 0.8013
Epoch 8/16
33/33 [======

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/410project/saved_model/cat_breed_classifier_model.no_munch_2')

In [ ]:
#Loading the model
model = load_model('/content/drive/MyDrive/410project/saved_model/cat_breed_classifier_model.no_munch_2')



In [ ]:
# Evaluate the Accuracy of the model
# validation_generator yields batches of images and their corresponding labels from the specified directory.IF we do not like how it works we can change it
test_loss, test_accuracy = model.evaluate(validation_generator, verbose = 2)

# Printing the accuracy
print('\nTest Accuracy:', test_accuracy)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


15/15 - 18s - loss: 0.3880 - accuracy: 0.8597 - 18s/epoch - 1s/step

Test Accuracy: 0.8596882224082947


In [ ]:
#Prediction
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(validation_generator)


15/15 [==============================] - 18s 1s/step


In [ ]:
#Preparing the Input Data

# Load and preprocess the input image in order for it to be the same way as the training set
img_path = "/content/drive/MyDrive/410project/test_images/testing_code.webp" #Add after test_images the actual image name
img = image.load_img(img_path, target_size=(250, 250))  # Resize image to match model input dimensions
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension



In [ ]:
#Interpreting Predictions
print(model.input_shape)
print(img_array.shape)

prediction_interpretation = model.predict(img_array)

(None, 250, 250, 3)
(1, 250, 250, 3)
1/1 [==============================] - 0s 53ms/step


In [ ]:
# Interpreting the prediction
predicted_class_index = np.argmax(prediction_interpretation) # argmax is used to find the index of the class with the highes probability.
predicted_class_probability = np.max(prediction_interpretation) #use max to show the one with the highest probability
class_labels = ["persian","abyssian","toyger"] # ,"munchkin"
predicted_class_label = class_labels[predicted_class_index]

In [ ]:
print('Predicted Class: ', predicted_class_label)
print('Predicted Probability: ', predicted_class_probability)

Predicted Class:  toyger
Predicted Probability:  0.9999975
